In [1]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sentence_transformers import SentenceTransformer
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, AdamW, GPT2Tokenizer, TFGPT2LMHeadModel, AutoTokenizer, AutoModelForCausalLM
import torch
import tensorflow as tf

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
import os
os.chdir("Y:\Python\script\Study\Dacon_0201")
os.getcwd()

'Y:\\Python\\script\\Study\\Dacon_0201'

In [4]:
df = pd.read_csv("./data/train_1on1_cat.csv")

In [5]:
df.head()

,category,질문,답변
0,건축구조,면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,건축구조,면진장치가 뭐야?,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을..."
2,건축구조,면진장치가 뭐야?,면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...
3,건축구조,면진장치가 뭐야?,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...
4,건축구조,면진장치가 뭐야?,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...


In [7]:
df['input_text'] = df['질문'] + tokenizer.eos_token + df['답변']
df['encoded_text'] = df['input_text'].apply(lambda x : tokenizer.encode(x, return_tensors='pt'))

In [8]:
df['category'].value_counts()

마감재       2720
인테리어      1230
시공        1110
마감하자       600
건축구조       310
기타         270
타 마감하자     200
Name: category, dtype: int64

In [9]:
tmp_df = df[df['category'] == '타 마감하자']

In [10]:
sample_arr = tmp_df['질문'].unique()

In [11]:
sample_arr[:5]

array(['외벽모서리 부위 결로의 대책은 뭐야?', '외벽 모서리 부위 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?',
       '결로가 발생하는 원인이 뭐야?', '결로가 발생하는 주된 이유가 무엇인가요?', '현관문 주위 결로의 대책은 뭐야?'],
      dtype=object)

In [12]:
model = AutoModelForCausalLM.from_pretrained("beomi/open-llama-2-ko-7b")
tokenizer = AutoTokenizer.from_pretrained("beomi/open-llama-2-ko-7b", padding_side = 'left')

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [13]:
from tqdm import tqdm

ans_li = []

model.to(device)
for i in tqdm(sample_arr):
    input_ids = tokenizer.encode(i + tokenizer.eos_token, return_tensors = 'pt')
    
    output_sequences = model.generate(
        input_ids=input_ids.to(device),
        max_length=300,
        temperature=0.9,
        top_k=1,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1)
    
    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        answer_start = full_text.find(tokenizer.eos_token) + len(tokenizer.eos_token)
        answer_only = full_text[answer_start:].strip()
        answer_only = answer_only.replace('\n', ' ')
        ans_li.append(answer_only)

  0%|                                                                                           | 0/40 [00:00<?, ?it/s]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
C:\Users\yjg10\anaconda3\lib\site-packages\transformers\models\llama\modeling_llama.py:728: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  2%|██                                                                                 | 1/40 [01:27<57:02, 87.75s/it]


KeyboardInterrupt: 